**1. Packages installation**

In [ ]:
#Import Package

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm
#from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import classification_report,accuracy_score,recall_score, f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.model_selection import  cross_val_score

In [ ]:
#pip install qiskit-machine-learning

In [ ]:
#pip install qiskit-algorithms

In [ ]:
#pip install qiskit_aer

In [ ]:
#pip install qiskit_ibm_runtime

In [ ]:
#pip install tensorflow

In [ ]:
#pip install torch torchvision torchaudio

**2. Read Input Dataset**

In [ ]:
#Read KIRC datasets

df= pd.read_csv("C:/Users/USER/NQE+QSVM/mRNA/KIRC_outliers_fix.csv")

df  

In [ ]:
#pwd

In [ ]:
import seaborn as sns
feat_df = df[['ENSG00000004939.12','ENSG00000070915.8','ENSG00000074803.16','ENSG00000086159.11','ENSG00000100362.11','ENSG00000102678.6','ENSG00000104413.14','Class Labels']]
sns.pairplot(feat_df, hue='Class Labels')

In [ ]:
#Number of data and column(502 data patients,48 gene features,1 class labels  )
print("The shape of the dataset :",df.shape)

In [ ]:
# Class label (0 = non metastasis(Negative), 1 = metastasis(Positive))

print("Number of data each class :")
print(df['Class Labels'].value_counts())

In [ ]:
#select 48 column gene(Features)
column_data = df.drop('Class Labels',axis=1)
#nc=df.drop(48,axis=0)
print ('48 features :',column_data.columns)
#print ('48 features :',nc.columns)

In [ ]:
#features are gene values each column (X)
#take the values each column gene
features = column_data.values
print("Feature values each column :\n",features)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# scale features
scaler = MinMaxScaler()
model=scaler.fit(features)
scaled_data=model.transform(features)
 
# print scaled features
print(scaled_data)

**3.PCA**

In [ ]:
#Compute Covariance Matrix
cov_matrix = np.cov(df, rowvar=False)
cov_matrix

In [ ]:
#Find Eigenvalues
eigenvalues, _ = np.linalg.eig(cov_matrix)
eigenvalues

In [ ]:
#Sort Eigenvalues
sorted_eigenvalues = np.sort(eigenvalues)[::-1]
sorted_eigenvalues

In [ ]:
#Compute sum of the first 9 eigenvalues
sum_first_9 = np.sum(sorted_eigenvalues[:9])
sum_first_9

In [ ]:
#calculate ratio for 9 eigenvalues
ratio_9 = sum_first_9 / np.sum(eigenvalues)
ratio_9

In [ ]:
#Compute sum of the first 12 eigenvalues
sum_first_12 = np.sum(sorted_eigenvalues[:12])
sum_first_12

In [ ]:
#Calculate ratio for 12 eigenvalues
ratio_12 = sum_first_12 / np.sum(eigenvalues)
ratio_12

In [ ]:
# Format results to 3 significant figures
formatted_ratio_9 = "{:.3g}".format(ratio_9)
formatted_ratio_12 = "{:.3g}".format(ratio_12)

# Print results
print("Ratio for first 9 eigenvalues:", formatted_ratio_9)
print("Ratio for first 12 eigenvalues:", formatted_ratio_12)

In [ ]:
C = np.cov(features, rowvar = False)
C

In [ ]:
#eignvalues

eigenvalues,eigenvectors = np.linalg.eig(C)
u, sigma, vt=np.linalg.svd(C)
print('eigenvalues :',eigenvalues)
print('eigenvectors :',eigenvectors)
print('sigma :',sigma)

In [ ]:
#Create dimension reduction from 48 features to 10 features
n_com=12
n_feat=n_com
pca=PCA(n_components=n_com)
principalComponents=pca.fit_transform(features)

In [ ]:
#save the 9PCs using dataframe
principalDf=pd.DataFrame(data=principalComponents,
                         columns=['PC1',
                                  'PC2',
                                  'PC3', 'PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12'])
finalDf=pd.concat([principalDf, df[['Class Labels']]],axis=1)
finalDf.head()
pd.plotting.parallel_coordinates(
finalDf, 'Class Labels', color=('#556270', '#4ECDC4'))

In [ ]:
#Print the new 9PCs dataframe
finalDf.head()

In [ ]:
#the shape of the new 9PCs dataframe
finalDf.shape

In [ ]:
# Splitting the dataset into the Training set and Test set
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
#
# X is 9 PCs and y is Class label (0 = non metastasis(Negative), 1 = metastasis(Positive))

X_PCA = finalDf.drop('Class Labels',axis=1).values
y_PCA = finalDf['Class Labels'].values

#if you use random_state=some_number(0,42,82),
#then we can guarantee that the spliting data output of Run 1 will be equal to the output of Run 2

#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_PCA,y_PCA, test_size=1/3,train_size=2/3,random_state=42)

print("X Training set :",X_train.shape)
print("y Training set :",y_train.shape)
print("X Test set :",X_test.shape)
print("y Test set :",y_test.shape)

print("X_train value :",X_train)
print("Y_train value :",y_train)
print("X_test value :",X_test)
print("Y_test value :",y_test)

In [ ]:

n_feat = n_com
n_train = y_train.shape
n_test = y_test.shape

**4. Imbalance Dataset Using SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
# Split the data to train and test dataset.


y_before_sm = Counter(y_train)
print('Number of data before applying SMOTE :',y_before_sm)
# oversampling the train dataset using SMOTE
smt = SMOTE(random_state=82,k_neighbors=5)
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

y_after_sm = Counter(y_train_sm)
print('Number of data after applying SMOTE :',y_after_sm)

**5. Classical SVM Classification**

In [ ]:
#Classification using data after smote and using default svm parameter
from sklearn import svm
svn_sm = svm.SVC()
svn_sm.fit(X_train_sm, y_train_sm)

In [ ]:
# Predict from the test dataset

pred_sm = svn_sm.predict(X_test)

# Calculate the accuracy
print("Accuracy after using SMOTE :",accuracy_score(y_test, pred_sm))

In [ ]:
# classification report
print("Classification after using SMOTE :\n")
print(classification_report(y_test, pred_sm))

print("F1 score:",f1_score(y_test, pred_sm))
print("Recall score:",recall_score(y_test, pred_sm))
print("Accuracy score:",accuracy_score(y_test, pred_sm))

In [ ]:
ConfusionMatrixDisplay.from_estimator(svn_sm, X_test, y_test)
plt.title("Confusion matrix after using SMOTE")
plt.show()

**6. Hyperparameter Tunning Classical SVC Classification**

In [ ]:
#Optimation C and gamma parameter
#reference: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/?ref=rp

#set the new for : C = [ 60, 80, 100, 120, 140], gamma = 10e-5 [1/4, 1/2 , 1, 2, 4]
#print out the accuracy each combination
# defining parameter range
param_grid = {'C': [1,20,40,60, 80, 100, 120, 140,160,180,200,210,215,220,225,230,235,240,260],
              'gamma': ['scale',(0.0001*1/4),(0.0001*1/2),(0.0001),(0.0001*2),(0.0001*4)],
             'kernel': ['rbf']}
from sklearn.svm import SVC

#param_grid = {'C': [ 0.00001, 0.0001, 0.001, 0.01, 0.1, 1,10,100,1000,10000],
#              'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1,10,100,1000,10000],
#              'kernel': ['rbf']}

#this code for print 'recall','f1','accuracy', set refit=false
#grid = GridSearchCV(SVC(), param_grid,scoring=['recall','f1','accuracy'], refit = False, verbose = 3)

#this code for search the best parameters
#set refit=true to outomatic select the best parameter and set to svm model
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,scoring='f1')


# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
#grid_predictions = grid.predict(X_test)
grid_predictions = grid.predict(X_test)

In [ ]:
# classification report
print("SVM Classification using optimal C and gamma parameter :\n")
print(classification_report(y_test, grid_predictions))

print("F1 score:",f1_score(y_test, grid_predictions))
print("Recall score:",recall_score(y_test, grid_predictions))
print("Accuracy score:",accuracy_score(y_test, grid_predictions))

In [ ]:
# Compute confusion matrix
cm2 = confusion_matrix(y_test, grid_predictions)
print("SVM Confusion matrix using optimal C and gamma parameter:\n")
print(cm2)

In [ ]:
ConfusionMatrixDisplay.from_estimator(grid, X_test, y_test)
plt.title("SVM Confusion matrix using optimal C and gamma parameter")
plt.show()

**7. QSVC**

In [ ]:
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler

fidelity = ComputeUncompute(sampler=Sampler())

In [ ]:
#create a new quantum kernel with the fidelity instance.
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel

feature_map = ZZFeatureMap(n_feat)
new_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

**8. NQE**

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from qiskit import *
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler as QiskitSampler
from qiskit_aer.noise import NoiseModel

from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

In [ ]:
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F

In [ ]:
#pip install torch torchvision torchaudio

In [ ]:
#pip install torch

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from qiskit import *
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler as QiskitSampler
from qiskit_aer.noise import NoiseModel

from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

In [ ]:
X_train= X_train_sm
Y_train= y_train_sm
X_test= X_test
Y_test= y_test

In [ ]:
def new_data(batch_size, X, Y):
    X1_new, X2_new, Y_new = [], [], []
    for i in range(batch_size):
        n, m = np.random.randint(len(X)), np.random.randint(len(X))
        X1_new.append(X[n])
        X2_new.append(X[m])
        if Y[n] == Y[m]:
            Y_new.append(1)
        else:
            Y_new.append(0)
    X1_new, X2_new, Y_new = torch.tensor(X1_new).to(torch.float32), torch.tensor(X2_new).to(torch.float32), torch.tensor(Y_new).to(torch.float32)
    return X1_new, X2_new, Y_new

X1_new_valid, X2_new_valid, Y_new_valid = new_data(10, X_test, Y_test)

In [ ]:
#pip install pennylane==0.27.0

In [ ]:
import pennylane as qml
from pennylane import numpy as np

N_layers = 3

# exp(ixZ) gate
def exp_Z(x, wires, inverse=False):
    if inverse == False:
        qml.RZ(-2 * x, wires=wires)
    elif inverse == True:
        qml.RZ(2 * x, wires=wires)

# exp(i(pi - x1)(pi - x2)ZZ) gate
def exp_ZZ2(x1, x2, wires, inverse=False):
    if inverse == False:
        qml.CNOT(wires=wires)
        qml.RZ(-2 * (np.pi - x1) * (np.pi - x2), wires=wires[1])
        qml.CNOT(wires=wires)
    elif inverse == True:
        qml.CNOT(wires=wires)
        qml.RZ(2 * (np.pi - x1) * (np.pi - x2), wires=wires[1])
        qml.CNOT(wires=wires)


# Quantum Embedding 1 for model 1 (Conventional ZZ feature embedding)
def QuantumEmbedding1(input):
    for i in range(N_layers):
        for j in range(n_feat):
            qml.Hadamard(wires=j)
            exp_Z(input[j], wires=j)
        for k in range(n_feat-1):
            exp_ZZ2(input[k], input[k+1], wires=[k,k+1])
        exp_ZZ2(input[n_feat-1], input[0], wires=[n_feat-1, 0])

def QuantumEmbedding1_inverse(input):
    for i in range(N_layers):
        exp_ZZ2(input[n_feat-1], input[0], wires=[n_feat-1, 0], inverse=True)
        for k in reversed(range(n_feat-1)):
            exp_ZZ2(input[k], input[k+1], wires=[k,k+1], inverse=True)
        qml.Barrier()
        for j in range(n_feat):
            exp_Z(input[j], wires=j, inverse=True)
            qml.Hadamard(wires=j)

dev = qml.device('default.qubit', wires=n_feat)

@qml.qnode(dev, interface="torch")
def circuit1(inputs):
    QuantumEmbedding1(inputs[0:n_feat])
    QuantumEmbedding1_inverse(inputs[n_feat:2 * n_feat])
    return qml.probs(wires=range(n_feat))

In [ ]:
from torch import nn
class Model1_Fidelity(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1 = qml.qnn.TorchLayer(circuit1, weight_shapes={})
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(n_feat, 2 * n_feat),
            nn.ReLU(),
            nn.Linear(2 * n_feat, 2 * n_feat),
            nn.ReLU(),
            nn.Linear(2 * n_feat, n_feat)
        )
    def forward(self, x1, x2):
        x1 = self.linear_relu_stack1(x1)
        x2 = self.linear_relu_stack1(x2)
        x = torch.concat([x1, x2], 1)
        x = self.qlayer1(x)
        return x[:,0]

model = Model1_Fidelity()

In [ ]:
# Define model, optimizer, and loss function
loss_fn = torch.nn.MSELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)
train_loss=[]
# Start training

loss_list = []  # Store loss history
model.train()  # Set model to training mode

iterations = 1000
for it in range(iterations):
    X1_batch, X2_batch, Y_batch = new_data(10, X_train, Y_train)
    pred = model(X1_batch, X2_batch)
    loss = loss_fn(pred, Y_batch)
    train_loss.append(loss.item())

    opt.zero_grad()
    loss.backward()
    opt.step()

    if it % 10 == 0:
        print(f"Iterations: {it} Loss: {loss.item()}")
        with torch.no_grad():
            pred_validation = model(X1_new_valid, X2_new_valid)
            loss_validation = loss_fn(pred_validation, Y_new_valid)
            print(f"Validation Loss: {loss_validation}")

In [ ]:
import pandas as pd

# Define model, optimizer, and loss function
loss_fn = torch.nn.MSELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.01)

train_loss = []
val_loss = []

# Start training
model.train()  # Set model to training mode

iterations = 1000
for it in range(iterations):
    X1_batch, X2_batch, Y_batch = new_data(10, X_train, Y_train)
    pred = model(X1_batch, X2_batch)
    loss = loss_fn(pred, Y_batch)
    train_loss.append(loss.item())

    opt.zero_grad()
    loss.backward()
    opt.step()

    if it % 10 == 0:
        print(f"Iterations: {it} Loss: {loss.item()}")
        with torch.no_grad():
            pred_validation = model(X1_new_valid, X2_new_valid)
            loss_validation = loss_fn(pred_validation, Y_new_valid)
            val_loss.append(loss_validation.item())
            print(f"Validation Loss: {loss_validation}")

# ===== After training: save to Excel =====
# Make val_loss same length as train_loss
# Fill missing validation losses with None
val_loss_full = [None] * iterations
for idx, val in enumerate(val_loss):
    val_loss_full[idx * 10] = val

# Create DataFrame
loss_data = {
    'Iteration': list(range(iterations)),
    'Training Loss': train_loss,
    'Validation Loss': val_loss_full
}
df_loss = pd.DataFrame(loss_data)

# Save to Excel
df_loss.to_excel('12_loss_results.xlsx', index=False)
print("Training and validation loss saved to '12_loss_results.xlsx'.")


In [ ]:
plt.plot(train_loss)

In [ ]:
torch.save(model.state_dict(), 'Model1.pt')

In [ ]:
class x_transform(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack1 = nn.Sequential(
            nn.Linear(n_feat, 2 * n_feat),
            nn.ReLU(),
            nn.Linear(2 * n_feat, 2 * n_feat),
            nn.ReLU(),
            nn.Linear(2 * n_feat, n_feat)
        )
    def forward(self, x):
        x = self.linear_relu_stack1(x)
        return x

model = x_transform()

In [ ]:
model.load_state_dict(torch.load('Model1.pt', map_location=torch.device('cpu')))

In [ ]:
X_train2 = torch.tensor(X_train).to(torch.float32)
X_test2 = torch.tensor(X_test).to(torch.float32)

In [ ]:
X_train_nn = model(X_train2)
X_test_nn = model(X_test2)

In [ ]:
X_train_nn2 = X_train_nn.detach().numpy()
X_test_nn2 = X_test_nn.detach().numpy()
print(X_train_nn2)


In [ ]:
import numpy as np
import pandas as pd

# Assuming X_train_nn2, X_test_nn2, y_train, and y_test are already defined


# Concatenate X and y for training and test data along the columns (axis=1)
train_data = np.concatenate((X_train_nn2, Y_train.reshape(-1, 1)), axis=1)
test_data = np.concatenate((X_test_nn2, Y_test.reshape(-1, 1)), axis=1)


# Convert the concatenated data into DataFrames
train_df = pd.DataFrame(train_data, columns=[f"feature_{i+1}" for i in range(X_train_nn2.shape[1])] + ["label"])
test_df = pd.DataFrame(test_data, columns=[f"feature_{i+1}" for i in range(X_test_nn2.shape[1])] + ["label"])
print(train_df.shape)
print(train_df["label"].value_counts())

train_df.to_excel("12PCs_X_train_with_labels.xlsx", index=False)
test_df.to_excel("12PCs_X_test_with_labels.xlsx", index=False)

print("Files saved as 'X_train_with_labels.xlsx' and 'X_test_with_labels.xlsx'")


**9. QSVC Classification and  Hyperparameter Tunning**

In [ ]:
# Define the original RBF kernel function
from functools import reduce

# Define the data mapping function using the original RBF kernel equation
#the data map function has 2 parameter input which is x and gamma. x is the input dataset and gamma is the value for rbf kernel
#the logic meaning is if the length of dataset=1 then set coeff to the value of data if more than 1 then do calculation
#reduce is use to sum all result of calculation
#lamdda is used to multiplied the value of dataset gamma
def data_map_eq10_ori(x, gamma):
    coeff = x[0] if len(x) == 1 else reduce(lambda m, n: np.exp(-gamma*((m - n)*(m - n))), x)
    return coeff

In [ ]:
from sklearn.model_selection import StratifiedKFold
splits=5
skf = StratifiedKFold(n_splits=splits)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score


In [ ]:
n_features = X_train_nn2.shape[1]
g_scale = 1 / (n_features * X_train_nn2.var())
g_scale

In [ ]:
from qiskit_machine_learning.algorithms import QSVC
from sklearn.metrics import confusion_matrix, f1_score
import math
import time
from itertools import product
import statistics
import pandas as pd

# Define the hyperparameters to search over
n_features = X_train_nn2.shape[1]
g_scale = 1 / (n_features * X_train_nn2.var())

gamma_values = [g_scale/10, g_scale/100, g_scale, g_scale + 50]
C_values = [0.1, 1, 10]
best_f1 = 0.0
best_gamma = None
best_C = None
start_time = time.time()

# Prepare a list to collect results
results = []

for gamma_val, C_val in product(gamma_values, C_values):
    data_map_func_rbf = lambda x: data_map_eq10_ori(x, gamma_val)
    rbf_feature_map = ZZFeatureMap(feature_dimension=n_feat, reps=2, entanglement='linear', data_map_func=data_map_func_rbf)
    new_kernel = FidelityQuantumKernel(feature_map=rbf_feature_map, fidelity=fidelity)
    qsvc = QSVC(quantum_kernel=new_kernel, C=C_val)

    F1_num = []  # Store F1 scores for each fold
    for i, (train_index, val_index) in enumerate(skf.split(X_train_nn2, Y_train)):
        X_train_fold, X_val_fold = X_train_nn2[train_index], X_train_nn2[val_index]
        Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

        # Train the QSVC classifier
        qsvc.fit(X_train_fold, Y_train_fold)

        # Predict the labels for the validation data
        predicted_labels = qsvc.predict(X_val_fold)

        # Confusion Matrix Calculation
        tn, fp, fn, tp = confusion_matrix(Y_val_fold, predicted_labels).ravel()
        print("tn:",tp,"fp:",fp,"fn:",fn,"tp:",tp)
        # Calculate metrics using provided formulas
        TPR = tp / (tp + fn)
        SPC = tn / (fp + tn)
        PPV = tp / (tp + fp)
        NPV = tn / (tn + fn) 
        FPR = fp / (fp + tn)
        FDR = fp / (fp + tp) 
        FNR = fn / (fn + tp) 
        ACC = (tp + tn) / (tn + fp + fn + tp) 
        F1 = 2 * tp / (2 * tp + fp + fn) 
        MCC = ((tp * tn) - (fp * fn)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) 

        # Append F1 score for this fold
        F1_num.append(F1)

        # Append metrics for this fold
        results.append({
            "Gamma": gamma_val,
            "C": C_val,
            "Fold": i + 1,
            "TP": tp, "TN": tn, "FP": fp, "FN": fn,
            "TPR": TPR, "SPC": SPC, "PPV": PPV, "NPV": NPV,
            "FPR": FPR, "FDR": FDR, "FNR": FNR, "ACC": ACC,
            "F1": F1, "MCC": MCC
        })

    # Calculate the average F1 score across folds
    avg_F1 = statistics.mean(F1_num)
    print(f"Gamma: {gamma_val}, C: {C_val}, Average F1 Score: {avg_F1}")

    # Track the best hyperparameters based on average F1 score
    if avg_F1 > best_f1:
        best_f1 = avg_F1
        best_gamma = gamma_val
        best_C = C_val

# Print the best hyperparameters and F1 score
print("Best gamma:", best_gamma)
print("Best C:", best_C)
print("Best Average F1 Score:", best_f1)

# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("qsvc_12PCs_cross_validation_results.csv", index=False)

end_time = time.time()
execution_time = end_time - start_time
print("Total execution time:", execution_time, "seconds")
print("Results saved to 'qsvc_12PCs_cross_validation_results.csv'")


In [ ]:
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler

fidelity = ComputeUncompute(sampler=Sampler())
#create a new quantum kernel with the fidelity instance.
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
n_feat=4
feature_map = ZZFeatureMap(n_feat)
new_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [ ]:
#QSVM+NN using best gamma and C by rbf kernel
from qiskit_machine_learning.algorithms import QSVC
gamma = best_gamma
#gamma = 9.475
#best_C=1
best_C=best_C
data_map_func_rbf = lambda x: data_map_eq10_ori(x, gamma)
rbf_feature_map = ZZFeatureMap(feature_dimension=n_feat, reps=2, entanglement='linear', data_map_func=data_map_func_rbf)
new_kernel = FidelityQuantumKernel(feature_map=rbf_feature_map, fidelity=fidelity)


qsvc = QSVC(quantum_kernel=new_kernel, C=best_C)


# Train the QSVC classifier
qsvc.fit(X_train_nn2, Y_train)

# Predict the labels for the test data
predicted_labels = qsvc.predict(X_test_nn2)

# Evaluate the classification accuracy
qsvc_score = qsvc.score(X_test_nn2, Y_test)

print(f"QSVM+NN classification test score: {qsvc_score}")

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print("QSVM+NN result:\n")
print(classification_report(y_test, predicted_labels))
print("F1 :",f1_score(y_test, predicted_labels))

In [ ]:
ConfusionMatrixDisplay.from_estimator(qsvc, X_test_nn2, Y_test)
plt.title("QSVM+NN Confusion matrix using optimal C and gamma parameter")
plt.show()

**10. Classical SVC Classification and Hyperparameter Tunning**

In [ ]:
#Optimation C and gamma parameter
#reference: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/?ref=rp

#set the new for : C = [ 60, 80, 100, 120, 140], gamma = 10e-5 [1/4, 1/2 , 1, 2, 4]
#print out the accuracy each combination
# defining parameter range
param_grid = {'C': [1,20,40,60, 80, 100, 120, 140,160,180,200,210,215,220,225,230,235,240,260],
              'gamma': ['scale',(0.0001*1/4),(0.0001*1/2),(0.0001),(0.0001*2),(0.0001*4)],
             'kernel': ['rbf']}
from sklearn.svm import SVC

#param_grid = {'C': [ 0.00001, 0.0001, 0.001, 0.01, 0.1, 1,10,100,1000,10000],
#              'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1,10,100,1000,10000],
#              'kernel': ['rbf']}

#this code for print 'recall','f1','accuracy', set refit=false
#grid = GridSearchCV(SVC(), param_grid,scoring=['recall','f1','accuracy'], refit = False, verbose = 3)

#this code for search the best parameters
#set refit=true to outomatic select the best parameter and set to svm model
grid_ml = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,scoring='f1')


# fitting the model for grid search
grid_ml.fit(X_train_nn2, Y_train)

In [ ]:
# print best parameter after tuning
print(grid_ml.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid_ml.best_estimator_)

In [ ]:
#grid_predictions = grid.predict(X_test)
grid_predictions_ml = grid_ml.predict(X_test_nn2)

In [ ]:
# classification report
print("SVM+NN Classification  :\n")
print(classification_report(y_test, grid_predictions_ml))

print("F1 score:",f1_score(y_test, grid_predictions_ml))
print("Recall score:",recall_score(y_test, grid_predictions_ml))
print("Accuracy score:",accuracy_score(y_test, grid_predictions_ml))

In [ ]:
# Compute confusion matrix
cm2_ml = confusion_matrix(y_test, grid_predictions_ml)
print("SVM+NN Confusion matrix using optimal C and gamma parameter:\n")
print(cm2_ml)

In [ ]:
from sklearn.svm import SVC  # Import classical SVC
from sklearn.metrics import confusion_matrix, f1_score
import math
import time
from itertools import product
import statistics
import pandas as pd

# Define the hyperparameters to search over
n_features = X_train_nn2.shape[1]
g_scale = 1 / (n_features * X_train_nn2.var())

gamma_values =[(0.0001*1/4),(0.0001*1/2),(0.0001),(0.0001*2),(0.0001*4),0.00001, 0.0001, 0.001, 0.01, 0.1, 1,10,100,500,1000,g_scale/10,g_scale/100,g_scale,g_scale+10,g_scale+20,g_scale+30,g_scale+40,g_scale+50]
C_values = [1,20,40,60,80,100,120,140,160,180,200,210,215,220,225,230,235,240,260]
best_f1 = 0.0
best_gamma = None
best_C = None
start_time = time.time()

# Prepare a list to collect results
results = []

for gamma_val, C_val in product(gamma_values, C_values):
    # Create classical SVC instead of QSVC
    svc = SVC(C=C_val, kernel='rbf', gamma=gamma_val)  # SVC with RBF kernel

    F1_num = []  # Store F1 scores for each fold
    for i, (train_index, val_index) in enumerate(skf.split(X_train_nn2, Y_train)):
        X_train_fold, X_val_fold = X_train_nn2[train_index], X_train_nn2[val_index]
        Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

        # Train the SVC classifier
        svc.fit(X_train_fold, Y_train_fold)

        # Predict the labels for the validation data
        predicted_labels = svc.predict(X_val_fold)

        # Confusion Matrix Calculation
        tn, fp, fn, tp = confusion_matrix(Y_val_fold, predicted_labels).ravel()
        print("tn:", tp, "fp:", fp, "fn:", fn, "tp:", tp)
        
        # Calculate metrics using provided formulas
        TPR = tp / (tp + fn)
        SPC = tn / (fp + tn)
        PPV = tp / (tp + fp)
        NPV = tn / (tn + fn)
        FPR = fp / (fp + tn)
        FDR = fp / (fp + tp)
        FNR = fn / (fn + tp)
        ACC = (tp + tn) / (tn + fp + fn + tp)
        F1 = 2 * tp / (2 * tp + fp + fn)
        MCC = ((tp * tn) - (fp * fn)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

        # Append F1 score for this fold
        F1_num.append(F1)

        # Append metrics for this fold
        results.append({
            "Gamma": gamma_val,
            "C": C_val,
            "Fold": i + 1,
            "TP": tp, "TN": tn, "FP": fp, "FN": fn,
            "TPR": TPR, "SPC": SPC, "PPV": PPV, "NPV": NPV,
            "FPR": FPR, "FDR": FDR, "FNR": FNR, "ACC": ACC,
            "F1": F1, "MCC": MCC
        })

    # Calculate the average F1 score across folds
    avg_F1 = statistics.mean(F1_num)
    print(f"Gamma: {gamma_val}, C: {C_val}, Average F1 Score: {avg_F1}")

    # Track the best hyperparameters based on average F1 score
    if avg_F1 > best_f1:
        best_f1 = avg_F1
        best_gamma = gamma_val
        best_C = C_val

# Print the best hyperparameters and F1 score
print("Best gamma:", best_gamma)
print("Best C:", best_C)
print("Best Average F1 Score:", best_f1)

# Save results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel("SVC_12PCs_cross_validation_results.xlsx", index=False)

end_time = time.time()
execution_time = end_time - start_time
print("Total execution time:", execution_time, "seconds")
print("Results saved to 'SVC_12PCs_cross_validation_results.xlsx'")
